In [1]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import seaborn as sns
from sklearn.ensemble import IsolationForest
import joblib
os.chdir('Resources/')

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('10_SP_Preprocessed_Data.csv')

X = df.drop(['HeartDisease'], axis='columns')
Y = df[['HeartDisease']]

In [3]:
from sklearn.ensemble import RandomForestClassifier
    
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1786)

rf = RandomForestClassifier(random_state=1397)
    
model_rf = rf.fit(X_train, Y_train)
    
pred_rf = model_rf.predict(X_test)
    
accuracy_rf = f1_score(Y_test, pred_rf)
    
print(accuracy_rf)

0.9456066945606695


In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import joblib

pca = PCA(n_components=0.95, random_state=1745)
rf_model_pca = RandomForestClassifier(random_state=1745, n_estimators=100)

X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(X, Y, test_size=0.2, random_state=159)

X_train_pca = pca.fit_transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)

rf_model_pca.fit(X_train_pca, Y_train_pca)
Y_pred_pca = rf_model_pca.predict(X_test_pca)

accuracy_rf_pca = f1_score(Y_test_pca, Y_pred_pca)

print(accuracy_rf_pca)

0.9385245901639344


In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import joblib

_if = IsolationForest(contamination=0.05, random_state=227, n_estimators=100, max_samples=256)

outlier_predictions_if = _if.fit_predict(X)

X_filtered_if = X[outlier_predictions_if != -1].reset_index(drop=True)
Y_filtered_if = Y[outlier_predictions_if != -1].reset_index(drop=True)

X_train_if, X_test_if, Y_train_if, Y_test_if = train_test_split(
    X_filtered_if, Y_filtered_if, test_size=0.2, random_state=227
)

rf_model_if = RandomForestClassifier(random_state=227, n_estimators=94, max_depth=18, criterion='gini')
rf_model_if.fit(X_train_if, Y_train_if)

Y_pred_if = rf_model_if.predict(X_test_if)

accuracy_if = f1_score(Y_test_if, Y_pred_if)

print(accuracy_if)

0.9421841541755889


In [6]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
    
X_train_rf_xgb, X_test_rf_xgb, Y_train_rf_xgb, Y_test_rf_xgb = train_test_split(X, Y, test_size=0.2, random_state=1786)

rf = RandomForestClassifier(random_state=1397, n_estimators=99, criterion='gini', 
                                max_features=3, max_depth=19, min_samples_split=2)
rf.fit(X_train_rf_xgb, Y_train_rf_xgb)
rf_pred = rf.predict(X_test_rf_xgb)
rf_f1 = f1_score(Y_test_rf_xgb, rf_pred)
    
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=1397)
xgb.fit(X_train_rf_xgb, Y_train_rf_xgb)
xgb_pred = xgb.predict(X_test_rf_xgb)
xgb_f1 = f1_score(Y_test_rf_xgb, xgb_pred)
    
combined_pred = []
for r, x in zip(rf_pred, xgb_pred):
    combined_pred.append(r if r == x else r)
    
combined_f1 = f1_score(Y_test_rf_xgb, combined_pred)
    
print(combined_f1)

0.9478079331941545


In [7]:
test_df = pd.DataFrame(X_test_rf_xgb)
test_df['Label'] = Y_test_rf_xgb.reset_index(drop=True)
test_df.to_csv('12_SP_Test_Data_RF_XGB.txt', index=False, sep='\t')

with open('12_SP_F1_Scores_RF_XGB.txt', 'w') as f:
    f.write(f"{combined_f1}\n")

In [8]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

iso_forest = IsolationForest(contamination=0.05, random_state=227, n_estimators=100, max_samples=256)
outliers = iso_forest.fit_predict(X)
X_filtered = X[outliers != -1].reset_index(drop=True)
Y_filtered = Y[outliers != -1].reset_index(drop=True)

X_train_rf_if_xgb, X_test_rf_if_xgb, Y_train_rf_if_xgb, Y_test_rf_if_xgb = train_test_split(
    X_filtered, Y_filtered, test_size=0.2, random_state=227
)

rf = RandomForestClassifier(random_state=227, n_estimators=94, max_depth=18, max_features=3)
rf.fit(X_train_rf_if_xgb, Y_train_rf_if_xgb)
rf_pred = rf.predict(X_test_rf_if_xgb)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=227)
xgb.fit(X_train_rf_if_xgb, Y_train_rf_if_xgb)
xgb_pred = xgb.predict(X_test_rf_if_xgb)

combined_pred = []
for r, x in zip(rf_pred, xgb_pred):
    combined_pred.append(r if r == x else r)

combined_f1 = f1_score(Y_test_rf_if_xgb, combined_pred)

print(combined_f1)

0.9421841541755889
